In [1]:
import pandas as pd

In [2]:
import os
print(os.getcwd())

C:\Users\manju\Downloads\codealpha_tasks-main\codealpha_tasks-main


In [11]:
df = pd.read_csv(r"spotify_millsongdata.csv")

In [12]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [13]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [14]:
df.shape

(57650, 4)

In [15]:
# checking for missing values
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [16]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [17]:
df.head(10)

,artist,song,text
0,Louis Armstrong,Christmas Night In Harlem,"Hallelujah, Hallelujah, glory, Halle \r\nWhat..."
1,John Denver,Bread And Roses,"This song appears on three albums, and was fir..."
2,Waylon Jennings,A Bad Day,I guess it all started when I woke up \r\nThi...
3,Deep Purple,No One Came,Maybe it's because I'm only starting \r\nThat...
4,Cinderella,The More Things Change,Woke up this morning on the wrong side of bed ...
5,Depeche Mode,The Things You Said,I heard it from my friends \r\nAbout the thin...
6,Alan Parsons Project,Let's Talk About Me,Let's talk about me for a minute \r\nWell how...
7,Regine Velasquez,Bluer Than Blue,"After you go, I can catch on my reading \r\nA..."
8,James Taylor,Get A Job,(Get a job) \r\n \r\nEvery morning about thi...
9,Doors,"Dead Cats, Dead Rats","Dead cats, dead rats \r\nCan't see what they ..."


In [18]:
df['text'][0]

"Hallelujah, Hallelujah, glory, Halle  \r\nWhat's all that noise?  \r\nHallelujah  \r\nDon't give up with that now  \r\n  \r\nGlory, Halle  \r\nWho's that over in the corner?  \r\nHallelujah  \r\nI believe that's Mr. Teagarden  \r\n  \r\nHey what you doin' up there, man?  \r\nLast time I saw you  \r\nYou were sayin' fare thee well to Harlem  \r\nI done bin wise g'wine  \r\nAn' I ain't g'wine there n'more  \r\n  \r\nYou sho' come back the right time  \r\nIt's Christmas up here  \r\nWhat d'ya mean up here?  \r\nIt's Christmas ev'rywhere  \r\n  \r\nYeah, but it ain't all Christmas  \r\nLike it's Christmas in Harlem  \r\nNo  \r\n  \r\nEv'ry gal struttin' with her beau  \r\nThrough the streets covered white with snow  \r\nHappy smiles er'ywhere you go  \r\nChristmas night in Harlem  \r\n  \r\nBlack and tans feelin' mighty good  \r\nIn that old colored neighborhood  \r\nHere and now be it understood  \r\nChristmas night in Harlem  \r\n  \r\nEv'ryone is gonna sit up  \r\nUntil after three  \r

In [19]:
df = df.sample(5000)

In [20]:
df.shape

(5000, 3)

# Text Cleaning/ Text Preprocessing

In [22]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [44]:
import nltk
nltk.download("punkt")
from nltk.stem.porter import PorterStemmer

In [45]:
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [46]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [49]:
similarity[0]

array([1.        , 0.07812543, 0.01529122, ..., 0.04856392, 0.01491452,
       0.04841804])

In [50]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [51]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [53]:
recommendation('Crying Over You')

In [54]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))